In [15]:
import pandas as pd
from glob import glob
from canvasapi import Canvas
import re

In [16]:
glob("marina/*")

['marina/Polls-per-user-Lecture0_09-24.xlsx',
 'marina/~$Polls-per-user-Lecture0_09-24.xlsx',
 'marina/students.csv']

In [1]:
# Canvas
API_URL = "https://canvas.ucsd.edu/"
API_KEY = ... # Account -> Settings -> New Access Token
COURSE_ID = 31376 
# Sooh DSC30 FA21: 31700
# Marina DSC30 FA21: 31376


# Sli.do results to be added
all_files = ['marina/Polls-per-user-Lecture0_09-24.xlsx']

In [18]:
# Login to Canvas
canvas = Canvas(API_URL, API_KEY)
course = canvas.get_course(COURSE_ID)
print(course.name)

DSC 30 - DataStrc/Algrthms for Data Sci - Langlois [FA21]


In [48]:
### Process Sli.do result
names = []
scores = []
for fname in all_files:
    name = re.findall("(Lecture[0-9]+)",fname)[0]
    names.append(name)
    raw = pd.read_excel(fname, header = 0, skiprows=[1])
    df = raw.drop(columns = ["User ID", "User Name", "User company", "Total Correct Answers"]).set_index("User Email")
    # drop question if only one person answer the question
    df = df.drop(columns = df.columns[df.notna().sum()<=1])
    scores.append((df.isna().mean(axis = 1) < 0.25).rename(name).to_frame())
print(names)

['Lecture0']


In [20]:
### Find assisgnment group
group_name = "Lecture Attendance"
group_id = -1
for group in course.get_assignment_groups():
    if group.name == group_name:
        group_id = group.id
        break
assert group_id != -1, "Manually create the assignemt group first"
group_id

136770

In [21]:
### Get student id - Name - Email
### Rerun when new student is added

# student_ls = []
# for stu in course.get_users(enrollment_type=['student'], enrollment_state=['active']):
#     student_ls.append([stu.id, stu.name, stu.get_profile()["primary_email"]])
# students = pd.DataFrame(student_ls, columns = ["id", "name", "email"])
# students.to_csv("marina/students.csv")
# print(len(students))

if COURSE_ID == 31700:
    students = pd.read_csv("sooh/students.csv", index_col=0)
elif COURSE_ID == 31376:
    students = pd.read_csv("marina/students.csv", index_col=0)
else:
    raise NotImplementedError()
print(len(students))

34


In [22]:
# Join 
joined = students.copy().set_index("email")
for score in scores:
    joined = joined.join(score, how="outer")
joined = joined.fillna(False)

In [23]:
joined.tail()

,id,name,Lecture0
z1song@ucsd.edu,21918.0,Zhengde Song,True
zgong@ucsd.edu,101994.0,Ziling Gong,True
zhw040@ucsd.edu,72683.0,Zhekai Wang,True
zhz044@ucsd.edu,94249.0,Zhuji Zhang,True
ziz013@ucsd.edu,81204.0,Sharon Zhao,True


In [25]:
existed_assignment = [a.name for a in course.get_assignments()]

In [26]:
for col in names:
    if col in existed_assignment:
        continue
    new_assignment = course.create_assignment({
        'name': col,
        'notify_of_update': False,
        'points_possible': 1,
        'published': True,
        "assignment_group_id":group_id
    })
    for email, id  in joined["id"][joined[col]].items():
        if id == False:
            print(email + "not on canvas")
            continue
        new_assignment.get_submission(id).edit(submission={'posted_grade': 1})
            

soohyunnamliao@gmail.com
